Endpoint raiz (metodo GET, retorna a página inicial)

In [ ]:
import requests

url = "http://localhost:5000/" 
resposta = requests.get(url)

print("Status:", resposta.status_code)
print("Conteúdo:")
print(resposta.text) 

Status: 200
Conteúdo:
<h1>Seja bem-vindo à Clínica! 🏥</h1>


Endpoint para listagem de pacientes (método GET)

In [19]:
url = "http://localhost:5000//pacientes"
response = requests.get(url)

print("Status:", response.status_code)
print("Resposta:", response.json())

Status: 200
Resposta: [{'cpf': '12345678900', 'data_nascimento': '1990-01-01', 'id': 1, 'nome': 'João', 'prontuario': 'queixas de dor de cabeça', 'telefone': '11999999999', 'tipo': 'paciente'}, {'cpf': '12395678932', 'data_nascimento': '1980-06-30', 'id': 2, 'nome': 'Vivian Marshall', 'prontuario': 'queixas de dor na lombar', 'telefone': '11979978868', 'tipo': 'paciente'}, {'cpf': '10987654321', 'data_nascimento': '1990-08-15', 'id': 3, 'nome': 'Maria Souza', 'prontuario': 'PRONT123', 'telefone': '21912345678', 'tipo': 'paciente'}]


Endpoint para lbusca de pacientes por ID (método GET)

In [13]:
url = "http://localhost:5000/pacientes/1"
response = requests.get(url)

print("Status:", response.status_code)
print("Resposta:", response.json())

Status: 200
Resposta: {'cpf': '12345678900', 'data_nascimento': '1990-01-01', 'id': 1, 'nome': 'João', 'prontuario': 'queixas de dor de cabeça', 'telefone': '11999999999', 'tipo': 'paciente'}


Endpoint para listagem de medicos (método GET)

In [14]:
url = "http://localhost:5000/medicos"
response = requests.get(url)

print("Status:", response.status_code)
print("Resposta:", response.json())

Status: 200
Resposta: [{'cpf': '12345678901', 'crm': 'CRM12345', 'data_nascimento': '1980-05-15', 'especialidade_id': 2, 'id': 1, 'nome': 'Dra. Ana Vargas', 'telefone': '11999998888', 'tipo': 'medico'}, {'cpf': '12395678901', 'crm': 'CRM12945', 'data_nascimento': '1980-12-15', 'especialidade_id': 2, 'id': 2, 'nome': 'Dra Carina', 'telefone': '11999978888', 'tipo': 'medico'}, {'cpf': '12395678931', 'crm': 'CRM12985', 'data_nascimento': '1980-12-30', 'especialidade_id': 3, 'id': 3, 'nome': 'Dr Geraldo', 'telefone': '11999978868', 'tipo': 'medico'}, {'cpf': '12345978901', 'crm': 'CRM12845', 'data_nascimento': '1980-05-20', 'especialidade_id': 1, 'id': 5, 'nome': 'Dr. João Silva', 'telefone': '21987654321', 'tipo': 'medico'}]


Endpoint para busca de medicos por ID (método GET)

In [15]:
url = "http://localhost:5000/medicos/1"
response = requests.get(url)

print("Status:", response.status_code)
print("Resposta:", response.json())

Status: 200
Resposta: {'cpf': '12345678901', 'crm': 'CRM12345', 'data_nascimento': '1980-05-15', 'especialidade_id': 2, 'id': 1, 'nome': 'Dra. Ana Vargas', 'telefone': '11999998888', 'tipo': 'medico'}


Endpoint para listagem de consultas (método GET)

In [22]:
url = "http://localhost:5000/consultas"
response = requests.get(url)

print("Status:", response.status_code)
print("Resposta:", response.json())

Status: 200
Resposta: [{'data_consulta': '2025-06-10', 'horario': '14:30', 'id': 1, 'medico_id': 1, 'paciente_id': 1}, {'data_consulta': '2025-06-10', 'horario': '14:30', 'id': 2, 'medico_id': 1, 'paciente_id': 1}, {'data_consulta': '2025-11-24', 'horario': '15:00', 'id': 3, 'medico_id': 2, 'paciente_id': 1}, {'data_consulta': '2025-10-24', 'horario': '15:00', 'id': 4, 'medico_id': 3, 'paciente_id': 2}, {'data_consulta': '2025-10-24', 'horario': '15:10', 'id': 5, 'medico_id': 3, 'paciente_id': 2}]


Endpoint para a buscar de consultas por ID (Metodo GET)

In [ ]:
url = "http://localhost:5000/consultas/1"
response = requests.get(url)

print("Status:", response.status_code)
print("Resposta:", response.json())

Status: 200
Resposta: [{'data_consulta': '2025-06-10', 'horario': '14:30', 'id': 1, 'medico_id': 1, 'paciente_id': 1}, {'data_consulta': '2025-06-10', 'horario': '14:30', 'id': 2, 'medico_id': 1, 'paciente_id': 1}, {'data_consulta': '2025-11-24', 'horario': '15:00', 'id': 3, 'medico_id': 2, 'paciente_id': 1}, {'data_consulta': '2025-10-24', 'horario': '15:00', 'id': 4, 'medico_id': 3, 'paciente_id': 2}, {'data_consulta': '2025-10-24', 'horario': '15:10', 'id': 5, 'medico_id': 3, 'paciente_id': 2}]


Endpoint para listagem de tratamentos (Metodo GET)

In [23]:
url = "http://localhost:5000/tratamentos"
response = requests.get(url)

print("Status:", response.status_code)
print("Resposta:", response.json())

Status: 200
Resposta: [{'consulta_id': 1, 'data_fim': '2025-06-17', 'data_inicio': '2025-06-10', 'descricao': 'Tratamento para dor de cabeça', 'id': 1}, {'consulta_id': 1, 'data_fim': '2025-06-17', 'data_inicio': '2025-06-10', 'descricao': 'Tratamento para dor de cabeça', 'id': 2}, {'consulta_id': 1, 'data_fim': '2025-10-24', 'data_inicio': '2025-10-12', 'descricao': 'tomar dipirona para a dor de cabeca', 'id': 3}, {'consulta_id': 2, 'data_fim': '2025-07-30', 'data_inicio': '2025-07-12', 'descricao': 'Tratamento de dermatomicose', 'id': 4}, {'consulta_id': 3, 'data_fim': '2025-07-07', 'data_inicio': '2025-06-12', 'descricao': 'Tratamento de dor no abdomen', 'id': 5}]


Endpoint para busca de tratamentos por ID (metodo GET)

In [24]:
url = "http://localhost:5000/tratamentos/1"
response = requests.get(url)

print("Status:", response.status_code)
print("Resposta:", response.json())

Status: 200
Resposta: {'consulta_id': 1, 'data_fim': '2025-06-17', 'data_inicio': '2025-06-10', 'descricao': 'Tratamento para dor de cabeça', 'id': 1}


Endpoint para listagem dos medicamentos (metodo GET)

In [25]:
url = "http://localhost:5000/medicamentos"
response = requests.get(url)

print("Status:", response.status_code)
print("Resposta:", response.json())

Status: 200
Resposta: [{'dosagem': 20.0, 'forma_apresentacao': 'Comprimido', 'id': 1, 'nome': 'itraconazol', 'qtd_estoque': 10}, {'dosagem': 20.0, 'forma_apresentacao': 'Comprimido', 'id': 2, 'nome': 'buscopan', 'qtd_estoque': 20}, {'dosagem': 10.0, 'forma_apresentacao': 'Comprimido', 'id': 3, 'nome': 'loratadina', 'qtd_estoque': 15}, {'dosagem': 10.0, 'forma_apresentacao': 'Comprimido', 'id': 4, 'nome': 'dipirona', 'qtd_estoque': 10}]


Endpoint para busca de tratamentos por iD (metodo GET)

In [26]:
url = "http://localhost:5000/tratamentos/1"
response = requests.get(url)

print("Status:", response.status_code)
print("Resposta:", response.json())

Status: 200
Resposta: {'consulta_id': 1, 'data_fim': '2025-06-17', 'data_inicio': '2025-06-10', 'descricao': 'Tratamento para dor de cabeça', 'id': 1}


Endpoint para listagem de posologias (metodo GET)

In [27]:
url = "http://localhost:5000/posologias"
response = requests.get(url)

print("Status:", response.status_code)
print("Resposta:", response.json())

Status: 200
Resposta: []


Endpoint de listagem de posologias pelo ID (Metodo GET)

In [ ]:
url = "http://localhost:5000/posologias/1"
response = requests.get(url)

print("Status:", response.status_code)
print("Resposta:", response.json())

Status: 200
Resposta: []


Endpoint para incerir um novo paciente (metodo POST)

In [65]:
import requests

url = "http://localhost:5000/pacientes"
dados = {
    "nome": "João da Silva Silveira",
    "cpf": "12345678908",
    "data_nascimento": "1998-12-14",
    "telefone": "24998989797",
    "prontuario": "queixas de dor na perna",
    "tipo": "paciente"
}

response = requests.post(url, json=dados)

print("Status:", response.status_code)
print("Resposta:", response.json())

Status: 201
Resposta: {'cpf': '12345678908', 'data_nascimento': '1998-12-14', 'id': 7, 'nome': 'João da Silva Silveira', 'prontuario': 'queixas de dor na perna', 'telefone': '24998989797', 'tipo': 'paciente'}


Endpoint para postagem de um novo medico (método POST)

In [66]:
import requests

url = "http://localhost:5000/medicos"
dados = {
    "nome": "João Savior",
    "cpf": "12345676912",
    "data_nascimento": "1998-11-14",
    "telefone": "24998789716",
    "crm": "134781",
    "especialidade_id": "2",
    "tipo": "medico"
}

response = requests.post(url, json=dados)

print("Status:", response.status_code)
print("Resposta:", response.json())

Status: 201
Resposta: {'cpf': '12345676912', 'crm': '134781', 'data_nascimento': '1998-11-14', 'especialidade_id': 2, 'id': 7, 'nome': 'João Savior', 'telefone': '24998789716', 'tipo': 'medico'}


Endpoint para postagem de uma nova consulta (Metodo POST)

In [44]:
import requests

url = "http://localhost:5000/consultas"
dados = {
    "paciente_id": "4",
    "medico_id": "5",
    "data_consulta": "2025-06-14",
    "horario": "10:00"
}

response = requests.post(url, json=dados)

print("Status:", response.status_code)
print("Resposta:", response.json())

Status: 201
Resposta: {'data_consulta': '2025-06-14', 'horario': '10:00', 'id': 6, 'medico_id': 5, 'paciente_id': 4}


Endpoint para a postagem de um novo tratamento (metodo POST)

In [45]:
import requests

url = "http://localhost:5000/tratamentos"
dados = {
    "consulta_id": 4,
    "data_fim": "2025-06-17",
    "data_inicio": "2025-06-10",
    "descricao": "Tratamento para dor na lombar",
}

response = requests.post(url, json=dados)

print("Status:", response.status_code)
print("Resposta:", response.json())



Status: 201
Resposta: {'consulta_id': 4, 'data_fim': '2025-06-17', 'data_inicio': '2025-06-10', 'descricao': 'Tratamento para dor na lombar', 'id': 6}


Endpoint para criação de um novo medicamento (metodo POST)

In [49]:
import requests

url = "http://localhost:5000/medicamentos"
dados = {
   "dosagem": 10.0,
   "forma_apresentacao": "xarope",
   "nome": "vick vaporube",
   "qtd_estoque": 5
}

response = requests.post(url, json=dados)

print("Status:", response.status_code)
print("Resposta:", response.json())

Status: 201
Resposta: {'dosagem': 10.0, 'forma_apresentacao': 'xarope', 'id': 6, 'nome': 'vick vaporube', 'qtd_estoque': 5}


Endpoint para postar uma nova posologia (Método POST)

In [50]:
import requests

url = "http://localhost:5000/posologias"
dados = {
   "medicamento_id": 2,
   "tratamento_id": 5,
   "receita": "tomar 1 comprimido a cada 6 horas por 7 dias",
   "qtd_utilizada": 1
}

response = requests.post(url, json=dados)

print("Status:", response.status_code)
print("Resposta:", response.json())

Status: 500


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Endpoint para editar um paciente (Método PUT-Por ID)

In [51]:
url = "http://localhost:5000/pacientes/1"
dados_atualizados = {
    "cpf": "12345678901",
    "data_nascimento": "1990-01-03",
    "nome": "Joao Teixeira",
    "prontuario": "queixas de dor de cabeca intensa",
    "telefone": "11999999996",
}

response = requests.put(url, json=dados_atualizados)

print(response.status_code)
print(response.json())

200
{'cpf': '12345678901', 'data_nascimento': '1990-01-03', 'id': 1, 'nome': 'Joao Teixeira', 'prontuario': 'queixas de dor de cabeca intensa', 'telefone': '11999999996', 'tipo': 'paciente'}


Endpoint para editar um medico (metodo PUT- Por ID)

In [52]:
url = "http://localhost:5000/medicos/1"
dados_atualizados = {
    "cpf": "12345678911",
    "data_nascimento": "1990-01-04",
    "nome": "Dra Ana Matias",
    "telefone": "11999959996",
    "CRM": "21167"
}

response = requests.put(url, json=dados_atualizados)

print(response.status_code)
print(response.json())

200
{'cpf': '12345678911', 'crm': 'CRM12345', 'data_nascimento': '1990-01-04', 'especialidade_id': 2, 'id': 1, 'nome': 'Dra Ana Matias', 'telefone': '11999959996', 'tipo': 'medico'}


Endpoint para edição de consultas (Metodo PUT- Por ID)

In [53]:
url = "http://localhost:5000/consultas/1"
dados_atualizados = {
    "data_consulta": "2025-10-25",
    "horario": "15:20",
    "id": 5,
    "medico_id": 3,
    "paciente_id": 2
}

response = requests.put(url, json=dados_atualizados)

print(response.status_code)
print(response.json())

405


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Endpoint para edição de tratamento (metodo PUT - Por ID)

In [55]:
url = "http://localhost:5000/tratamentos/1"
dados_atualizados = {
    "consulta_id": 3,
    "data_fim": "2025-07-01",
    "data_inicio": "2025-06-12",
    "descricao": "Tratamento de dor no abdomen",
}

response = requests.put(url, json=dados_atualizados)

print(response.status_code)
print(response.json())

200
{'consulta_id': 3, 'data_fim': '2025-07-01', 'data_inicio': '2025-06-12', 'descricao': 'Tratamento de dor no abdomen', 'id': 1}


Endpoint para edição de Medicamentos (Metodo PUT - por ID)

In [56]:
url = "http://localhost:5000/medicamentos/1"
dados_atualizados = {
    "dosagem": 5.0,
    "forma_apresentacao": "xarope",
    "nome": "vick vaporube",
    "qtd_estoque": 7
}

response = requests.put(url, json=dados_atualizados)

print(response.status_code)
print(response.json())

200
{'dosagem': 5.0, 'forma_apresentacao': 'xarope', 'id': 1, 'nome': 'vick vaporube', 'qtd_estoque': 7}


Endpoint para edição das posologias (metodo PUT-por ID)

In [58]:
url = "http://localhost:5000/posologias/1"
dados_atualizados = {
    "medicamento_id": 1,
    "tratamento_id": 5,
    "receita": "tomar 1 comprimido a cada 6 horas por 7 dias",
    "qtd_utilizada": 1
}

response = requests.put(url, json=dados_atualizados)

print(response.status_code)
print(response.json())

405


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Endpoint para deletar um paciente (metodo DELETE - por ID)

In [67]:
url = "http://localhost:5000/pacientes/5"

response = requests.delete(url)

print("Status:", response.status_code)
print("Resposta:", response.text)

Status: 200
Resposta: {
  "mensagem": "Deletado com sucesso"
}



Endpoint para deletar um medico (metodo DELETE - por ID)

In [72]:
url = "http://localhost:5000/medicos/6"

response = requests.delete(url)

print("Status:", response.status_code)
print("Resposta:", response.text)

Status: 200
Resposta: {
  "mensagem": "Deletado com sucesso"
}



Endpoint para deletar uma consulta (metodo DELETE - por ID)

In [77]:
url = "http://localhost:5000/consultas/5"

response = requests.delete(url)

print("Status:", response.status_code)
print("Resposta:", response.text)

Status: 200
Resposta: {
  "mensagem": "Paciente deletado com sucesso"
}



Endpoint para deletar um tratamento (metodo DELETE - por ID)

In [78]:
url = "http://localhost:5000/tratamentos/5"

response = requests.delete(url)

print("Status:", response.status_code)
print("Resposta:", response.text)

Status: 200
Resposta: {
  "mensagem": "Deletado com sucesso"
}



Endpoint para deletar um medicamento (metodo DELETE - por ID)

In [84]:
url = "http://localhost:5000/medicamentos/4"

response = requests.delete(url)

print("Status:", response.status_code)
print("Resposta:", response.text)

Status: 200
Resposta: {
  "mensagem": "Deletado com sucesso"
}



Endpoint para deletar uma posologia (metodo DELETE - por ID)

In [80]:
url = "http://localhost:5000/posologias/5"

response = requests.delete(url)

print("Status:", response.status_code)
print("Resposta:", response.text)

Status: 500
Resposta: <!doctype html>
<html lang=en>
  <head>
    <title>TypeError: &#39;NoneType&#39; object is not callable
 // Werkzeug Debugger</title>
    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css">
    <link rel="shortcut icon"
        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">
    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>
    <script>
      var CONSOLE_MODE = false,
          EVALEX = true,
          EVALEX_TRUSTED = false,
          SECRET = "BYfMgkqTEhtg7UzieeIM";
    </script>
  </head>
  <body style="background-color: #fff">
    <div class="debugger">
<h1>TypeError</h1>
<div class="detail">
  <p class="errormsg">TypeError: &#39;NoneType&#39; object is not callable
</p>
</div>
<h2 class="traceback">Traceback <em>(most recent call last)</em></h2>
<div class="traceback">
  <h3></h3>
  <ul><li><div class="frame" id="frame-2696560725584">
  <h4>File <cite class="filename">"C:\Users\User\Doc